In [1]:
from config.configuration import RunDetails

# runDetail = RunDetails('config.yml', 'ASVspoof-2019_training')
runDetail = RunDetails('config-mfcc.yml', 'ASVspoof-2019_training_mfcc')

notebookName = 'audio-deepfake-detection-bulk-training'
random_state_lowValue = 1
random_state_highValue = 200

In [2]:
configFilename = runDetail.configFilename
runJobId = runDetail.jobId

In [3]:
import config.configuration as configuration
import model_definitions.model_cnn_definition as model_cnn_definition
from preprocessors.preprocessor_factory import PreprocessorFactory
from notebook_utils import notebookToPython
from processors.basic_model_training_processor import BasicModelTrainingProcessor
from processors.basic_model_evaluation_processor import BasicModelEvaluationProcessor
from processors.bulk_model_training_processor import BulkModelTrainingProcessor

In [4]:
config = configuration.ConfigLoader(configFilename)

notebookToPython(notebookName)
job = config.getJobConfig(runJobId)

if (job.newModelGenerated == False):
    raise ValueError("This notebook is meant for training. Select a job without a value for 'persisted-model' set.")

Write python file
Generating new model name: output/ASVspoof-2019_training_mfcc_2025-03-29T17-36-18.258353.libjob
Assigned model name: output/ASVspoof-2019_training_mfcc_2025-03-29T17-36-18.258353.libjob


In [5]:
preproc_factory = PreprocessorFactory()
preprocessor = preproc_factory.newPreprocessor(job.preprocessor)

MelFrequencyCepstralCoeffiecient


In [6]:
X, y_encoded = preprocessor.extract_features_multipleSource(job, job.dataPathSuffix)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt...
fullDataPath: C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac
Loading audio files: 1269
Loading audio files: 2538
Loading audio files: 3807
Loading audio files: 5076
Loading audio files: 6345
Loading audio files: 7614
Loading audio files: 8883
Loading audio files: 10152
Loading audio files: 11421
Loading audio files: 12690
Loading audio files: 13959
Loading audio files: 15228
Loading audio files: 16497
Loading audio files: 17766
Loading audio files: 19035
Loading audio files: 20304
Loading audio files: 21573
Loading audio files: 22842
Loading audio files: 24111
Loading audio files: 25380
Number of audio files load: 25380


In [7]:
bulkTrainingProc = BulkModelTrainingProcessor(job,
                                              model_cnn_definition.ModelCnnDefinition,
                                              BasicModelTrainingProcessor,
                                              BasicModelEvaluationProcessor)

bulkTrainingProc.process(random_state_lowValue, random_state_highValue, X, y_encoded, 1)

Selecting training and test data - traininSplitRandomState: 1
Training using 20304 files.
    def buildModel(self) -> Model:
        print(f"__job__: {self.__job__}")
        input_shape = (self.__job__.numMels, self.width, self.channels)
        model_input = Input(shape = input_shape)
        job = self.__job__

        # Derived from:
        # Anagha, R., Arya, A., Narayan, V. H., Abhishek, S., & Anjali, T. (2023).
        #   Audio Deepfake Detection Using Deep Learning.
        #   2023 12th International Conference on System Modeling & Advancement in Research Trends (SMART), System Modeling & Advancement in Research Trends (SMART), 2023 12th International Conference On, 176–181.
        #   https://doi.org/10.1109/SMART59791.2023.10428163
        x = Conv2D(filters=32, kernel_size=job.kernelSize, activation='relu')(model_input)
        x = MaxPooling2D(pool_size=job.poolSize)(x)
        x = Conv2D(filters=64, kernel_size=job.kernelSize, activation='relu')(x)
        x = MaxPooli